In [ ]:
# [Nur Colab] Diese Zellen müssen nur auf *Google Colab* ausgeführt werden und installieren Packete und Daten
!wget -q https://raw.githubusercontent.com/KI-Campus/AMALEA/master/requirements.txt && pip install --quiet -r requirements.txt
!wget --quiet "https://github.com/KI-Campus/AMALEA/releases/download/data/data.zip" && unzip -q data.zip
!wget --quiet "https://github.com/KI-Campus/AMALEA/releases/download/images/images.zip" && unzip -q images.zip

# 🔧 Setup: Transfer Learning Libraries

import warnings
warnings.filterwarnings('ignore')

# Standard Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import os
import random
from typing import List, Tuple, Dict

# Deep Learning
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, optimizers, callbacks
from tensorflow.keras.applications import (
    ResNet50, VGG16, MobileNetV2, EfficientNetB0, 
    DenseNet121, Xception, InceptionV3
)
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Computer Vision
import cv2
from scipy import signal, ndimage

# Interactive Widgets
from ipywidgets import interact, widgets
from IPython.display import display, HTML

# Streamlit (für Apps)
import streamlit as st

# Plotting Configuration
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Seeds for Reproducibility
np.random.seed(42)
tf.random.set_seed(42)
random.seed(42)

print("🔄 Transfer Learning Setup abgeschlossen!")
print(f"📊 TensorFlow: {tf.__version__}")
print(f"🔢 NumPy: {np.__version__}")

# GPU Check
if tf.config.list_physical_devices('GPU'):
    print("🚀 GPU verfügbar für Training!")
else:
    print("💻 CPU wird verwendet")

# Memory Optimization für GPU
if tf.config.list_physical_devices('GPU'):
    try:
        for gpu in tf.config.experimental.list_physical_devices('GPU'):
            tf.config.experimental.set_memory_growth(gpu, True)
        print("🔧 GPU Memory Growth aktiviert")
    except RuntimeError as e:
        print(f"⚠️  GPU Konfiguration: {e}")

# Available Pre-trained Models
AVAILABLE_MODELS = {
    'ResNet50': ResNet50,
    'VGG16': VGG16,
    'MobileNetV2': MobileNetV2,
    'EfficientNetB0': EfficientNetB0,
    'DenseNet121': DenseNet121,
    'Xception': Xception,
    'InceptionV3': InceptionV3
}

print(f"\n🏗️ Verfügbare Pre-trained Models: {list(AVAILABLE_MODELS.keys())}")
print("✅ Bereit für Transfer Learning Experimente!")

# 🔄 06.4 Transfer Learning - Von Riesen auf die Schultern steigen

**Data Analytics & Big Data - Woche 6.4**  
*IU Internationale Hochschule*

---

## 🎯 Lernziele

Nach diesem Notebook können Sie:
- ✅ **Transfer Learning** verstehen und strategisch einsetzen
- ✅ **Pre-trained Models** (ResNet, EfficientNet, etc.) effektiv nutzen
- ✅ **Fine-tuning Strategien** für verschiedene Anwendungsfälle entwickeln
- ✅ **Feature Extraction vs. Fine-tuning** optimal auswählen
- ✅ **CIFAR-10 Performance** dramatisch verbessern mit wenig Training
- ✅ **Streamlit-App** für interaktive Model-Vergleiche erstellen

---

## 🤔 Was ist Transfer Learning?

**Transfer Learning** = Nutzen von bereits trainierten Modellen für neue Aufgaben

### 💡 Die Grundidee:

**Anstatt von Grund auf zu trainieren:**
```
Random Weights → Train for weeks → Hope for good results
```

**Transfer Learning Ansatz:**
```
Pre-trained Model → Fine-tune for hours → Excellent results
```

### 🏗️ "Standing on the Shoulders of Giants"

Große Modelle wurden bereits auf **Millionen von Bildern** trainiert:
- **ImageNet:** 14 Millionen Bilder, 1000 Klassen
- **JFT-300M:** 300 Millionen Bilder (Google)
- **OpenAI CLIP:** 400 Millionen Bild-Text Paare

Diese Modelle haben bereits gelernt:
- **Low-level Features:** Edges, Textures, Shapes
- **Mid-level Features:** Objektteile, Patterns
- **High-level Features:** Komplexe Objektmerkmale

## Autoencoder für semantische Segmentierung

### Semantische Segmentierung

Auf CNN basierende Modelle wurden in großer Vielfalt aufgebaut, um verschiedene Aufgaben zu lösen. Allgemein lassen sich die Herausforderungen der Klassifizierung, der semantischen Segmentierung, Objekterkennung und Instanzensegmentierung unter komplexeren neueren wie Keypoint Detection oder DensePose etc benennen.
Die Zuweisung einer Objektklasse, die in einem Bild als Ganzes eine Objektklasse zuzuordnen ist, wird als Klassifizierung bezeichnet. Während bei der semantischen Segmentierung alle Pixel durch die Objektklassen, auf die sie sich beziehen, identifiziert werden müssen. Im Gegensatz zur Klassifizierung können mehrere Objektklassen in einem Bild vorkommen.



### Segnet - Ein Autoencoder für semantische Segmentierung

Basierend auf [Kitti Road dataset](http://www.cvlibs.net/datasets/kitti/eval_road.php). Ein Segmentierungsdatensatz für autonomes Fahren, der vom __Karlsruher Institut für Technologie (KIT)__, dem MPI Tübingen und der University of Toronto erstellt wurde.


![CNN Autoencoder](images/segnet.png "CNN Autoencoder")


                                    Quelle: http://mi.eng.cam.ac.uk/projects/segnet/

Es ist möglich, diese Klassifizierungsaufgabe zu lösen, indem man am Ende eine Softmax-Schicht verwendet oder ein gegebenes RGB-Bild regressiert. Im letzteren Fall sind die RGB-Werte möglicherweise nicht genau gleich und es gibt eine intrinsische Ordnung in den Klassen. Auch wenn anschließend ein Schwellenwert verwendet wird, ist der Dimensionsraum allerdings viel kleiner. Im Allgemeinen sollte man sich für den ersten Ansatz entscheiden, da er das Problem als reguläre Klassifikationsaufgabe löst und gängige Praxis ist. Es ist nicht empfehlenswert, dies in einer Regression anzuwenden. Der zweite Ansatz dient nur dazu, alternative Wege zu zeigen, wie man ein Problem angehen kann (und zum Spaß).

Die Netzwerkarchitektur eines Autoencoders verwendet eine Struktur, die oft vorher auf einigen Daten wie [ImageNet](http://www.image-net.org/) trainiert wurde. Die Idee ist, dass diese Gewichte bereits etwas mit der späteren Aufgabe gemeinsam haben, so dass das Training schneller und möglicherweise besser konvergiert, als wenn man mit zufälligen Gewichten anfängt. In der obigen SegNet-Architektur wird die Standard-Klassifikationsnetzarchitektur `VGG-16` verwendet, um das Inputbild in einen höheren abstrakten Raum zu kodieren. Anschließend projizieren Upsampling und Faltungen die extrahierten Features zurück in den ursprünglichen Inputraum.

## 🎯 Transfer Learning Strategien

### 1. 🔒 Feature Extraction (Frozen Features)

**Wann verwenden:** Kleiner neuer Datensatz, ähnlich zu ImageNet

```python
# Base Model einfrieren
base_model.trainable = False
```

**Vorteile:**
- ⚡ Sehr schnelles Training
- 💾 Wenig GPU-Speicher benötigt
- 🛡️ Keine Gefahr der Feature-Zerstörung

**Nachteile:**
- 🎯 Features passen möglicherweise nicht perfekt zur neuen Aufgabe

### 2. 🔄 Fine-tuning (Trainable Features)

**Wann verwenden:** Größerer neuer Datensatz, unterschiedlich zu ImageNet

```python
# Base Model auftauen nach Initial Training
base_model.trainable = True
# Sehr niedrige Learning Rate verwenden!
optimizer = Adam(learning_rate=0.0001)
```

**Vorteile:**
- 🎯 Features werden optimal an neue Aufgabe angepasst
- 📈 Oft beste Performance

**Nachteile:**
- ⏱️ Längeres Training
- ⚠️ Risiko des Overfittings

### 3. 🎛️ Layer-wise Fine-tuning

**Strategie:** Verschiedene Teile des Netzwerks unterschiedlich behandeln

```python
# Frühe Layer einfrieren (generelle Features)
for layer in base_model.layers[:-50]:
    layer.trainable = False

# Späte Layer fine-tunen (spezifische Features)  
for layer in base_model.layers[-50:]:
    layer.trainable = True
```

### 📊 Entscheidungsmatrix

| Datensatz Größe | Ähnlichkeit zu ImageNet | Empfohlene Strategie |
|-----------------|-------------------------|---------------------|
| Klein | Hoch | Feature Extraction |
| Klein | Niedrig | Fine-tuning (wenige Layer) |
| Groß | Hoch | Fine-tuning |
| Groß | Niedrig | Fine-tuning (alle Layer) |

### 🚀 Warum funktioniert Transfer Learning so gut?

1. **Hierarchical Feature Learning:** CNNs lernen hierarchische Repräsentationen
2. **Domain Similarity:** Viele Computer Vision Tasks teilen grundlegende Features
3. **Computational Efficiency:** Nutzt Jahre an Forschung und GPU-Zeit
4. **Better Initialization:** Besserer Startpunkt als Random Weights

<div class="alert alert-block alert-success">
<b>Frage 5.4.1:</b> In welcher Stadt wurden die Bilder dieses Datensatzes erstellt?
</div>

<div class="alert alert-block alert-success">
<b>Ihre Antwort:</b></div>

## 🧠 Praxisprojekt: CIFAR-10 mit Transfer Learning

### 🎯 Ziel: Dramatische Performance-Verbesserung

**Baseline (aus Notebook 6.3):**
- Selbst trainiertes CNN: ~70-75% Accuracy
- Training: 10+ Epochen nötig

**Transfer Learning Ziel:**
- Pre-trained Model: 90%+ Accuracy  
- Training: 2-3 Epochen ausreichend

### 📊 Experimentaufbau

Wir werden verschiedene Ansätze vergleichen:

1. **🔒 Feature Extraction:** ResNet50 frozen + neue Classifier
2. **🔄 Fine-tuning:** ResNet50 trainable mit niedrigerer Learning Rate  
3. **🚀 Modern Architecture:** EfficientNet mit optimiertem Fine-tuning
4. **⚡ Lightweight:** MobileNetV2 für mobile/embedded Anwendungen

### 💡 Warum CIFAR-10 für Transfer Learning?

- **Realistic Challenge:** 32×32 Bilder sind kleiner als ImageNet (224×224)
- **Domain Gap:** Natürliche Objekte, aber andere Auflösung
- **Perfect Testbed:** Schnell zu trainieren, aber aussagekräftige Ergebnisse

### 🔧 Technical Challenges

1. **Input Size Adaptation:** CIFAR-10 (32×32) vs ImageNet (224×224)
2. **Output Layer Replacement:** 1000 ImageNet Klassen → 10 CIFAR-10 Klassen
3. **Learning Rate Scheduling:** Balance zwischen Speed und Stability
4. **Data Augmentation:** Optimale Kombination für kleine Bilder


# 📊 CIFAR-10 Dataset für Transfer Learning

print("📥 Lade CIFAR-10 Dataset...")

# CIFAR-10 laden
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Dataset Info
print("✅ CIFAR-10 erfolgreich geladen!")
print(f"\n📊 Dataset Übersicht:")
print(f"   Training: {x_train.shape} Bilder, {y_train.shape} Labels")
print(f"   Test: {x_test.shape} Bilder, {y_test.shape} Labels")
print(f"   Bildformat: {x_train.shape[1:]} (Height × Width × Channels)")

# Klassen definieren
num_classes = 10
classes = [
    'Flugzeug', 'Auto', 'Vogel', 'Katze', 'Hirsch',
    'Hund', 'Frosch', 'Pferd', 'Schiff', 'LKW'
]

print(f"\n🏷️  Klassen ({num_classes}):")
for i, class_name in enumerate(classes):
    class_count = np.sum(y_train == i)
    print(f"   {i}: {class_name} ({class_count:,} Trainingsbilder)")

# Daten für Transfer Learning vorbereiten
print(f"\n🔧 Vorbereitung für Transfer Learning...")

# 1. Normalisierung (0-255 → 0-1)
x_train_norm = x_train.astype('float32') / 255.0
x_test_norm = x_test.astype('float32') / 255.0

# 2. Labels zu kategorischen Vektoren
y_train_cat = keras.utils.to_categorical(y_train, num_classes)
y_test_cat = keras.utils.to_categorical(y_test, num_classes)

# 3. Input Size für Transfer Learning (32x32 → 224x224)
def resize_for_transfer_learning(images):
    """
    Resize CIFAR-10 images (32x32) to ImageNet size (224x224)
    """
    resized_images = np.zeros((len(images), 224, 224, 3))
    for i, img in enumerate(images):
        resized_images[i] = tf.image.resize(img, [224, 224])
    return resized_images

print("🔄 Resize für Transfer Learning (32×32 → 224×224)...")
x_train_resized = resize_for_transfer_learning(x_train_norm)
x_test_resized = resize_for_transfer_learning(x_test_norm)

print(f"   Original: {x_train_norm.shape}")
print(f"   Resized: {x_train_resized.shape}")

# Visualisierung: Original vs Resized
fig, axes = plt.subplots(2, 5, figsize=(15, 6))

for i in range(5):
    # Original 32x32
    axes[0, i].imshow(x_train_norm[i])
    axes[0, i].set_title(f'Original 32×32\n{classes[y_train[i][0]]}')
    axes[0, i].axis('off')
    
    # Resized 224x224 (showing scaled down for visualization)
    axes[1, i].imshow(x_train_resized[i])
    axes[1, i].set_title(f'Resized 224×224\n{classes[y_train[i][0]]}')
    axes[1, i].axis('off')

plt.suptitle('🔄 CIFAR-10: Original vs Transfer Learning Ready', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

print("✅ CIFAR-10 Transfer Learning Preparation abgeschlossen!")
print(f"📊 Resized Training Set: {x_train_resized.shape}")
print(f"📊 Categorical Labels: {y_train_cat.shape}")


In [ ]:
from PIL import Image
import matplotlib.pyplot as plt

# 🏗️ Pre-trained Models Exploration

print("🏗️ Exploring Pre-trained Models...")

def explore_pretrained_model(model_name, model_class):
    """
    Explore architecture and parameters of a pre-trained model
    """
    print(f"\n🔍 {model_name} Analysis:")
    print("=" * 50)
    
    # Load model without top layer
    model = model_class(
        weights='imagenet',
        include_top=False,
        input_shape=(224, 224, 3)
    )
    
    # Model statistics
    total_params = model.count_params()
    print(f"   📊 Total Parameters: {total_params:,}")
    print(f"   🏗️  Layers: {len(model.layers)}")
    print(f"   📏 Output Shape: {model.output_shape}")
    
    # Memory estimation (rough)
    memory_mb = (total_params * 4) / (1024**2)  # 4 bytes per float32
    print(f"   💾 Estimated Memory: {memory_mb:.1f} MB")
    
    return model, total_params

# Explore different architectures
model_stats = {}

# 1. ResNet50 - Residual Networks
resnet50, resnet_params = explore_pretrained_model("ResNet50", ResNet50)
model_stats['ResNet50'] = resnet_params

# 2. EfficientNetB0 - Efficient Architecture
efficientnet, efficient_params = explore_pretrained_model("EfficientNetB0", EfficientNetB0)
model_stats['EfficientNetB0'] = efficient_params

# 3. MobileNetV2 - Mobile-optimized
mobilenet, mobile_params = explore_pretrained_model("MobileNetV2", MobileNetV2)
model_stats['MobileNetV2'] = mobile_params

# Model Comparison Visualization
print("\n📊 Model Comparison:")
print("=" * 60)

models = list(model_stats.keys())
params = list(model_stats.values())

# Bar plot
plt.figure(figsize=(12, 6))
bars = plt.bar(models, params, color=['#FF6B6B', '#4ECDC4', '#45B7D1'])
plt.title('🏗️ Pre-trained Model Parameter Comparison', fontsize=14, fontweight='bold')
plt.ylabel('Parameters (millions)')
plt.xticks(rotation=45)

# Add value labels on bars
for bar, param in zip(bars, params):
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height + height*0.01,
             f'{param/1e6:.1f}M', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

# Feature Map Visualization Function
def visualize_feature_maps(model, sample_image, model_name, num_maps=8):
    """
    Visualize feature maps from different layers
    """
    # Get outputs from intermediate layers
    layer_names = [layer.name for layer in model.layers[::len(model.layers)//4]][:4]
    
    # Create model that outputs feature maps
    outputs = [model.get_layer(name).output for name in layer_names]
    feature_model = tf.keras.Model(inputs=model.input, outputs=outputs)
    
    # Get feature maps
    feature_maps = feature_model.predict(np.expand_dims(sample_image, axis=0))
    
    # Plot
    fig, axes = plt.subplots(len(layer_names), num_maps, figsize=(16, 8))
    
    for layer_idx, (layer_name, feature_map) in enumerate(zip(layer_names, feature_maps)):
        for map_idx in range(min(num_maps, feature_map.shape[-1])):
            ax = axes[layer_idx, map_idx] if len(layer_names) > 1 else axes[map_idx]
            
            # Normalize feature map for visualization
            fmap = feature_map[0, :, :, map_idx]
            fmap = (fmap - fmap.min()) / (fmap.max() - fmap.min() + 1e-8)
            
            ax.imshow(fmap, cmap='viridis')
            ax.axis('off')
            
            if map_idx == 0:
                ax.set_ylabel(f'{layer_name}', rotation=90, fontsize=10)
    
    plt.suptitle(f'🔍 {model_name} Feature Maps', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()

# Visualize feature maps for ResNet50
print("\n🎨 Feature Map Visualization (ResNet50):")
sample_image = x_train_resized[42]  # Pick a sample
visualize_feature_maps(resnet50, sample_image, "ResNet50")

print("✅ Pre-trained Model Exploration completed!")
print("\n💡 Key Insights:")
print("   • ResNet50: Balanced performance/size, good for most tasks")
print("   • EfficientNetB0: Best accuracy/parameter ratio")  
print("   • MobileNetV2: Lightweight, perfect for mobile deployment")
print("   • All models learn hierarchical features from simple to complex")

Als erstes wurde die Objekterkennung eingeführt. Ein Objektdetektor versucht, verschiedene, vordefinierte Objekte im Bild zu lokalisieren und zu klassifizieren. Dabei werden Bounding Boxes verwendet. Sie geben die Position des erkannten Objekts im Bild an. Zusätzlich wurde der Bounding Box ein Klassenlabel zugeordnet.

# 🔒 Strategie 1: Feature Extraction (Frozen Base Model)

print("🔒 Implementiere Feature Extraction Approach...")

def create_feature_extraction_model(base_model_class, model_name):
    """
    Erstellt Feature Extraction Model mit gefrorener Base
    """
    print(f"\n🏗️ Erstelle {model_name} Feature Extraction Model...")
    
    # 1. Pre-trained Base Model laden (ohne Top)
    base_model = base_model_class(
        weights='imagenet',
        include_top=False,
        input_shape=(224, 224, 3)
    )
    
    # 2. Base Model einfrieren - KEINE UPDATES während Training!
    base_model.trainable = False
    
    # 3. Custom Classifier Head hinzufügen
    inputs = tf.keras.Input(shape=(224, 224, 3))
    x = base_model(inputs, training=False)  # Explicitly set training=False
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    x = tf.keras.layers.Dense(128, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    outputs = tf.keras.layers.Dense(num_classes, activation='softmax')(x)
    
    model = tf.keras.Model(inputs, outputs)
    
    # 4. Model kompilieren mit normaler Learning Rate
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    # Statistiken
    total_params = model.count_params()
    trainable_params = sum([tf.keras.backend.count_params(w) for w in model.trainable_weights])
    frozen_params = total_params - trainable_params
    
    print(f"   📊 Total Parameters: {total_params:,}")
    print(f"   🔒 Frozen Parameters: {frozen_params:,} ({frozen_params/total_params*100:.1f}%)")
    print(f"   🎯 Trainable Parameters: {trainable_params:,} ({trainable_params/total_params*100:.1f}%)")
    
    return model

# Feature Extraction Models erstellen
print("🏗️ Erstelle verschiedene Feature Extraction Models...")

# ResNet50 Feature Extraction
resnet_fe = create_feature_extraction_model(ResNet50, "ResNet50")

# EfficientNet Feature Extraction  
efficient_fe = create_feature_extraction_model(EfficientNetB0, "EfficientNetB0")

# Kompakte Datensets für schnelles Training
print("\n📦 Erstelle kompakte Trainingssets...")

# Kleinere Subsets für Demo (in Production würde man alle Daten nutzen)
subset_size = 5000
test_subset_size = 1000

# Random indices für reproduzierbare Subsets
train_indices = np.random.choice(len(x_train_resized), subset_size, replace=False)
test_indices = np.random.choice(len(x_test_resized), test_subset_size, replace=False)

x_train_subset = x_train_resized[train_indices]
y_train_subset = y_train_cat[train_indices]
x_test_subset = x_test_resized[test_indices]
y_test_subset = y_test_cat[test_indices]

print(f"   Training Subset: {x_train_subset.shape}")
print(f"   Test Subset: {x_test_subset.shape}")

# Training Callbacks
callbacks_fe = [
    tf.keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=2),
]

# Feature Extraction Training
print("\n🚀 Trainiere Feature Extraction Models...")

# ResNet50 Training
print("\n1️⃣ ResNet50 Feature Extraction Training:")
resnet_fe_history = resnet_fe.fit(
    x_train_subset, y_train_subset,
    epochs=5,
    batch_size=32,
    validation_data=(x_test_subset, y_test_subset),
    callbacks=callbacks_fe,
    verbose=1
)

# EfficientNet Training
print("\n2️⃣ EfficientNetB0 Feature Extraction Training:")
efficient_fe_history = efficient_fe.fit(
    x_train_subset, y_train_subset,
    epochs=5,
    batch_size=32,
    validation_data=(x_test_subset, y_test_subset),
    callbacks=callbacks_fe,
    verbose=1
)

# Performance Evaluation
resnet_fe_loss, resnet_fe_acc = resnet_fe.evaluate(x_test_subset, y_test_subset, verbose=0)
efficient_fe_loss, efficient_fe_acc = efficient_fe.evaluate(x_test_subset, y_test_subset, verbose=0)

print("\n📊 Feature Extraction Results:")
print("=" * 50)
print(f"ResNet50:        {resnet_fe_acc:.4f} ({resnet_fe_acc*100:.2f}%)")
print(f"EfficientNetB0:  {efficient_fe_acc:.4f} ({efficient_fe_acc*100:.2f}%)")

# Training History Visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Accuracy comparison
ax1.plot(resnet_fe_history.history['val_accuracy'], label='ResNet50', marker='o')
ax1.plot(efficient_fe_history.history['val_accuracy'], label='EfficientNetB0', marker='s')
ax1.set_title('🔒 Feature Extraction: Validation Accuracy', fontweight='bold')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Accuracy')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Loss comparison
ax2.plot(resnet_fe_history.history['val_loss'], label='ResNet50', marker='o')
ax2.plot(efficient_fe_history.history['val_loss'], label='EfficientNetB0', marker='s')
ax2.set_title('🔒 Feature Extraction: Validation Loss', fontweight='bold')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Loss')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("✅ Feature Extraction Training abgeschlossen!")
print("\n💡 Erkenntnisse:")
print("   • Sehr schnelles Training (nur Classifier wird trainiert)")
print("   • Gute Performance trotz gefrorener Features")
print("   • Wenig GPU-Speicher benötigt")
print("   • Ideal für kleine Datensätze und schnelle Prototypen")

In [ ]:
# 🔄 Strategie 2: Fine-tuning (Trainable Base Model)

print("🔄 Implementiere Fine-tuning Approach...")

def create_fine_tuning_model(base_model_class, model_name, unfreeze_layers=50):
    """
    Erstellt Fine-tuning Model mit partial unfrozen Base
    """
    print(f"\n🏗️ Erstelle {model_name} Fine-tuning Model...")
    
    # 1. Pre-trained Base Model laden
    base_model = base_model_class(
        weights='imagenet',
        include_top=False,
        input_shape=(224, 224, 3)
    )
    
    # 2. Erst alle einfrieren
    base_model.trainable = False
    
    # 3. Model mit Classifier erstellen
    inputs = tf.keras.Input(shape=(224, 224, 3))
    x = base_model(inputs, training=False)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    x = tf.keras.layers.Dense(128, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    outputs = tf.keras.layers.Dense(num_classes, activation='softmax')(x)
    
    model = tf.keras.Model(inputs, outputs)
    
    # 4. Initial Compilation und kurzes Training (Feature Extraction Phase)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    print(f"   📚 Phase 1: Feature Extraction Training...")
    initial_history = model.fit(
        x_train_subset, y_train_subset,
        epochs=2,  # Kurz, nur zur Stabilisierung
        batch_size=32,
        validation_data=(x_test_subset, y_test_subset),
        verbose=0
    )
    
    # 5. Base Model für Fine-tuning auftauen
    base_model.trainable = True
    
    # 6. Nur die letzten Layer auftauen (frühe Layer bleiben gefroren)
    if unfreeze_layers > 0:
        for layer in base_model.layers[:-unfreeze_layers]:
            layer.trainable = False
    
    # 7. Re-compile mit SEHR niedriger Learning Rate!
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),  # 10x niedriger!
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    # Statistiken nach Fine-tuning Setup
    total_params = model.count_params()
    trainable_params = sum([tf.keras.backend.count_params(w) for w in model.trainable_weights])
    frozen_params = total_params - trainable_params
    
    print(f"   📊 Total Parameters: {total_params:,}")
    print(f"   🔒 Frozen Parameters: {frozen_params:,} ({frozen_params/total_params*100:.1f}%)")
    print(f"   🔄 Trainable Parameters: {trainable_params:,} ({trainable_params/total_params*100:.1f}%)")
    print(f"   🎯 Unfrozen Layers: {unfreeze_layers}")
    
    return model, initial_history

# Fine-tuning Models erstellen
print("🏗️ Erstelle Fine-tuning Models...")

# ResNet50 Fine-tuning
resnet_ft, resnet_initial = create_fine_tuning_model(ResNet50, "ResNet50", unfreeze_layers=50)

# EfficientNet Fine-tuning
efficient_ft, efficient_initial = create_fine_tuning_model(EfficientNetB0, "EfficientNetB0", unfreeze_layers=30)

# Fine-tuning Training mit besonderen Callbacks
print("\n🚀 Starte Fine-tuning Phase...")

callbacks_ft = [
    tf.keras.callbacks.EarlyStopping(
        patience=4, 
        restore_best_weights=True,
        monitor='val_accuracy'
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        factor=0.2, 
        patience=2,
        min_lr=1e-7,
        monitor='val_accuracy'
    ),
]

# ResNet50 Fine-tuning
print("\n1️⃣ ResNet50 Fine-tuning Phase:")
resnet_ft_history = resnet_ft.fit(
    x_train_subset, y_train_subset,
    epochs=8,
    batch_size=16,  # Kleinere Batch Size für Fine-tuning
    validation_data=(x_test_subset, y_test_subset),
    callbacks=callbacks_ft,
    verbose=1
)

# EfficientNet Fine-tuning
print("\n2️⃣ EfficientNetB0 Fine-tuning Phase:")
efficient_ft_history = efficient_ft.fit(
    x_train_subset, y_train_subset,
    epochs=8,
    batch_size=16,
    validation_data=(x_test_subset, y_test_subset),
    callbacks=callbacks_ft,
    verbose=1
)

# Performance Evaluation
resnet_ft_loss, resnet_ft_acc = resnet_ft.evaluate(x_test_subset, y_test_subset, verbose=0)
efficient_ft_loss, efficient_ft_acc = efficient_ft.evaluate(x_test_subset, y_test_subset, verbose=0)

print("\n📊 Fine-tuning Results:")
print("=" * 50)
print(f"ResNet50:        {resnet_ft_acc:.4f} ({resnet_ft_acc*100:.2f}%)")
print(f"EfficientNetB0:  {efficient_ft_acc:.4f} ({efficient_ft_acc*100:.2f}%)")

# Comprehensive Comparison: Feature Extraction vs Fine-tuning
print("\n📈 Complete Comparison:")
print("=" * 60)
print("Method               Model           Accuracy    Improvement")
print("=" * 60)
print(f"Feature Extraction   ResNet50        {resnet_fe_acc:.4f}      -")
print(f"Fine-tuning          ResNet50        {resnet_ft_acc:.4f}      +{((resnet_ft_acc/resnet_fe_acc)-1)*100:.1f}%")
print(f"Feature Extraction   EfficientNetB0  {efficient_fe_acc:.4f}      -")
print(f"Fine-tuning          EfficientNetB0  {efficient_ft_acc:.4f}      +{((efficient_ft_acc/efficient_fe_acc)-1)*100:.1f}%")
print("=" * 60)

# Visualization
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))

# 1. Accuracy Comparison Bar Chart
methods = ['ResNet50\nFeature Ext.', 'ResNet50\nFine-tuning', 
           'EfficientNet\nFeature Ext.', 'EfficientNet\nFine-tuning']
accuracies = [resnet_fe_acc, resnet_ft_acc, efficient_fe_acc, efficient_ft_acc]
colors = ['#FF6B6B', '#FF8E8E', '#4ECDC4', '#70D4C4']

bars = ax1.bar(methods, accuracies, color=colors)
ax1.set_title('🏆 Method Comparison: Accuracy', fontweight='bold')
ax1.set_ylabel('Test Accuracy')
ax1.set_ylim(0, 1)

for bar, acc in zip(bars, accuracies):
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height + 0.01,
             f'{acc:.3f}\n({acc*100:.1f}%)', ha='center', va='bottom', fontweight='bold')

# 2. Training History - ResNet50
ax2.plot(resnet_fe_history.history['val_accuracy'], label='Feature Extraction', marker='o')
ax2.plot(resnet_ft_history.history['val_accuracy'], label='Fine-tuning', marker='s')
ax2.set_title('📈 ResNet50: Training Progress', fontweight='bold')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Validation Accuracy')
ax2.legend()
ax2.grid(True, alpha=0.3)

# 3. Training History - EfficientNet
ax3.plot(efficient_fe_history.history['val_accuracy'], label='Feature Extraction', marker='o')
ax3.plot(efficient_ft_history.history['val_accuracy'], label='Fine-tuning', marker='s')
ax3.set_title('📈 EfficientNetB0: Training Progress', fontweight='bold')
ax3.set_xlabel('Epoch')
ax3.set_ylabel('Validation Accuracy')
ax3.legend()
ax3.grid(True, alpha=0.3)

# 4. Performance vs Parameters Trade-off
models_names = ['ResNet50', 'EfficientNetB0']
fe_accs = [resnet_fe_acc, efficient_fe_acc]
ft_accs = [resnet_ft_acc, efficient_ft_acc]

x_pos = np.arange(len(models_names))
width = 0.35

ax4.bar(x_pos - width/2, fe_accs, width, label='Feature Extraction', color='#FF6B6B', alpha=0.8)
ax4.bar(x_pos + width/2, ft_accs, width, label='Fine-tuning', color='#4ECDC4', alpha=0.8)

ax4.set_title('🎯 Architecture Comparison', fontweight='bold')
ax4.set_ylabel('Test Accuracy')
ax4.set_xticks(x_pos)
ax4.set_xticklabels(models_names)
ax4.legend()
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("✅ Fine-tuning Experiments abgeschlossen!")
print("\n💡 Key Insights:")
print(f"   • Fine-tuning verbessert Performance um {((max(resnet_ft_acc, efficient_ft_acc)/max(resnet_fe_acc, efficient_fe_acc))-1)*100:.1f}%")
print("   • Niedrige Learning Rate ist KRITISCH für Fine-tuning")
print("   • Layer-wise Unfreezing verhindert Feature-Zerstörung")
print("   • EfficientNet zeigt beste Accuracy/Parameter Ratio")

# 🎮 Interactive Transfer Learning Explorer

def interactive_transfer_learning_explorer():
    """
    🎮 Interaktiver Widget für Transfer Learning Parameter-Exploration
    """
    print("🎮 Interactive Transfer Learning Explorer")
    print("🔧 Experimentieren Sie mit verschiedenen Transfer Learning Strategien!")
    
    # Widget-Steuerungen
    model_selector = widgets.Dropdown(
        options=['ResNet50', 'EfficientNetB0', 'MobileNetV2'],
        value='ResNet50',
        description='Base Model:',
        style={'description_width': 'initial'}
    )
    
    strategy_selector = widgets.Dropdown(
        options=['Feature Extraction', 'Fine-tuning', 'Gradual Unfreezing'],
        value='Feature Extraction',
        description='Strategy:',
        style={'description_width': 'initial'}
    )
    
    learning_rate = widgets.FloatLogSlider(
        value=0.001,
        base=10,
        min=-5, # 1e-5
        max=-1, # 1e-1
        step=0.1,
        description='Learning Rate:',
        style={'description_width': 'initial'}
    )
    
    unfreeze_layers = widgets.IntSlider(
        value=50,
        min=0,
        max=100,
        step=10,
        description='Unfreeze Layers:',
        style={'description_width': 'initial'}
    )
    
    batch_size = widgets.Dropdown(
        options=[16, 32, 64],
        value=32,
        description='Batch Size:',
        style={'description_width': 'initial'}
    )
    
    def predict_performance(model_name, strategy, lr, unfreeze, batch):
        """
        Predict expected performance based on parameters
        (Simplified simulation for educational purposes)
        """
        # Base performance lookup
        base_performances = {
            'ResNet50': 0.85,
            'EfficientNetB0': 0.88,
            'MobileNetV2': 0.82
        }
        
        base_perf = base_performances[model_name]
        
        # Strategy adjustments
        if strategy == 'Feature Extraction':
            strategy_bonus = 0.0
        elif strategy == 'Fine-tuning':
            strategy_bonus = 0.03
        else:  # Gradual Unfreezing
            strategy_bonus = 0.05
        
        # Learning rate adjustment
        if lr > 0.01:
            lr_penalty = -0.02  # Too high
        elif lr < 0.0001:
            lr_penalty = -0.01  # Too low
        else:
            lr_penalty = 0.01   # Good range
        
        # Unfreeze layers adjustment (for fine-tuning)
        if strategy != 'Feature Extraction':
            if unfreeze < 20:
                unfreeze_adj = -0.01  # Too few
            elif unfreeze > 80:
                unfreeze_adj = -0.02  # Too many
            else:
                unfreeze_adj = 0.01   # Good range
        else:
            unfreeze_adj = 0
        
        # Batch size adjustment
        batch_adj = 0.005 if batch == 32 else 0 # 32 is often optimal
        
        # Calculate final performance
        final_perf = base_perf + strategy_bonus + lr_penalty + unfreeze_adj + batch_adj
        final_perf = max(0.5, min(1.0, final_perf))  # Clamp to realistic range
        
        return final_perf
    
    def update_transfer_learning(model_name, strategy, lr, unfreeze, batch):
        """Update Transfer Learning basierend auf Widget-Werten"""
        
        # Predicted Performance
        predicted_acc = predict_performance(model_name, strategy, lr, unfreeze, batch)
        
        # Performance Visualization
        fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 10))
        
        # 1. Predicted Accuracy Gauge
        ax1.pie([predicted_acc, 1-predicted_acc], 
                labels=[f'Predicted Accuracy\n{predicted_acc:.1%}', ''],
                colors=['#4ECDC4', '#E0E0E0'],
                startangle=90,
                counterclock=False)
        ax1.set_title('🎯 Predicted Performance', fontweight='bold')
        
        # 2. Strategy Comparison
        strategies = ['Feature\nExtraction', 'Fine-tuning', 'Gradual\nUnfreezing']
        strategy_accs = [
            predict_performance(model_name, 'Feature Extraction', lr, unfreeze, batch),
            predict_performance(model_name, 'Fine-tuning', lr, unfreeze, batch),
            predict_performance(model_name, 'Gradual Unfreezing', lr, unfreeze, batch)
        ]
        
        colors = ['#FF6B6B' if s.replace('\n', ' ') == strategy else '#E0E0E0' for s in strategies]
        bars = ax2.bar(strategies, strategy_accs, color=colors)
        ax2.set_title('📊 Strategy Comparison', fontweight='bold')
        ax2.set_ylabel('Predicted Accuracy')
        ax2.set_ylim(0.7, 1.0)
        
        # Highlight selected strategy
        for bar, acc in zip(bars, strategy_accs):
            if bar.get_facecolor()[:3] != (0.8784313725490196, 0.8784313725490196, 0.8784313725490196):  # Not gray
                height = bar.get_height()
                ax2.text(bar.get_x() + bar.get_width()/2., height + 0.005,
                        f'{acc:.3f}', ha='center', va='bottom', fontweight='bold')
        
        # 3. Model Architecture Visualization
        model_sizes = {'ResNet50': 25.6, 'EfficientNetB0': 5.3, 'MobileNetV2': 3.5}
        model_names = list(model_sizes.keys())
        sizes = list(model_sizes.values())
        colors_model = ['#4ECDC4' if m == model_name else '#E0E0E0' for m in model_names]
        
        bars = ax3.bar(model_names, sizes, color=colors_model)
        ax3.set_title('🏗️ Model Size (Million Parameters)', fontweight='bold')
        ax3.set_ylabel('Parameters (M)')
        
        # 4. Training Configuration
        config_data = {
            'Learning Rate': f'{lr:.1e}',
            'Batch Size': str(batch),
            'Strategy': strategy,
            'Unfreeze Layers': str(unfreeze) if strategy != 'Feature Extraction' else 'N/A'
        }
        
        ax4.axis('off')
        table_data = [[k, v] for k, v in config_data.items()]
        table = ax4.table(cellText=table_data,
                         colLabels=['Parameter', 'Value'],
                         loc='center',
                         cellLoc='left')
        table.auto_set_font_size(False)
        table.set_fontsize(10)
        table.scale(1, 2)
        ax4.set_title('⚙️ Configuration Summary', fontweight='bold', pad=20)
        
        plt.tight_layout()
        plt.show()
        
        # Recommendations
        print(f"\n💡 Recommendations for {model_name} with {strategy}:")
        
        if strategy == 'Feature Extraction':
            print("   ✅ Fast training, good for small datasets")
            print("   ✅ Lower computational requirements")
            print("   ⚠️  May not adapt perfectly to your domain")
        elif strategy == 'Fine-tuning':
            print("   ✅ Better adaptation to your specific task")
            print("   ✅ Usually achieves higher accuracy")
            print("   ⚠️  Requires careful learning rate tuning")
        else:  # Gradual Unfreezing
            print("   ✅ Best of both worlds approach")
            print("   ✅ Reduces risk of catastrophic forgetting")
            print("   ⚠️  More complex training procedure")
        
        # Parameter-specific advice
        if lr > 0.01:
            print("   🔴 Learning rate too high - may cause instability")
        elif lr < 0.0001:
            print("   🔴 Learning rate too low - training may be very slow")
        else:
            print("   ✅ Learning rate in good range")
    
    # Interactive Widget
    interact(update_transfer_learning,
             model_name=model_selector,
             strategy=strategy_selector,
             lr=learning_rate,
             unfreeze=unfreeze_layers,
             batch=batch_size)

# Widget anzeigen
interactive_transfer_learning_explorer()

In [ ]:
# 🚀 Advanced Transfer Learning Techniques

print("🚀 Advanced Transfer Learning Strategies...")

class AdvancedTransferLearning:
    """
    🏆 Advanced Transfer Learning Implementation
    
    Features:
    - Gradual Unfreezing
    - Progressive Learning Rates
    - Layer-wise Learning Rates
    - Discriminative Fine-tuning
    """
    
    def __init__(self, base_model_class, num_classes=10):
        self.base_model_class = base_model_class
        self.num_classes = num_classes
        self.model = None
        self.base_model = None
        
    def create_model(self):
        """Create model with advanced architecture"""
        # Base model
        self.base_model = self.base_model_class(
            weights='imagenet',
            include_top=False,
            input_shape=(224, 224, 3)
        )
        
        # Advanced classifier head
        inputs = tf.keras.Input(shape=(224, 224, 3))
        x = self.base_model(inputs, training=False)
        
        # Advanced pooling and regularization
        x = tf.keras.layers.GlobalAveragePooling2D()(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Dropout(0.3)(x)
        
        # Multi-layer classifier with residual connections
        x1 = tf.keras.layers.Dense(512, activation='relu')(x)
        x1 = tf.keras.layers.BatchNormalization()(x1)
        x1 = tf.keras.layers.Dropout(0.3)(x1)
        
        x2 = tf.keras.layers.Dense(256, activation='relu')(x1)
        x2 = tf.keras.layers.BatchNormalization()(x2)
        x2 = tf.keras.layers.Dropout(0.2)(x2)
        
        # Skip connection
        x = tf.keras.layers.Concatenate()([x1, x2])
        
        outputs = tf.keras.layers.Dense(self.num_classes, activation='softmax')(x)
        
        self.model = tf.keras.Model(inputs, outputs)
        return self.model
    
    def gradual_unfreezing_training(self, x_train, y_train, x_val, y_val):
        """
        🔄 Gradual Unfreezing Strategy
        
        Phase 1: Feature Extraction
        Phase 2: Unfreeze top layers
        Phase 3: Unfreeze all layers with very low LR
        """
        histories = []
        
        # Phase 1: Feature Extraction
        print("\n📚 Phase 1: Feature Extraction (All frozen)")
        self.base_model.trainable = False
        
        self.model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )
        
        history1 = self.model.fit(
            x_train, y_train,
            epochs=3,
            batch_size=32,
            validation_data=(x_val, y_val),
            verbose=1
        )
        histories.append(('Feature Extraction', history1))
        
        # Phase 2: Partial Unfreezing
        print("\n🔓 Phase 2: Partial Unfreezing (Top 30 layers)")
        self.base_model.trainable = True
        
        # Freeze early layers
        for layer in self.base_model.layers[:-30]:
            layer.trainable = False
        
        self.model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )
        
        history2 = self.model.fit(
            x_train, y_train,
            epochs=3,
            batch_size=32,
            validation_data=(x_val, y_val),
            verbose=1
        )
        histories.append(('Partial Unfreezing', history2))
        
        # Phase 3: Full Fine-tuning
        print("\n🎯 Phase 3: Full Fine-tuning (Very low LR)")
        
        # Unfreeze all layers
        for layer in self.base_model.layers:
            layer.trainable = True
        
        self.model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001),
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )
        
        history3 = self.model.fit(
            x_train, y_train,
            epochs=4,
            batch_size=16,  # Smaller batch for stability
            validation_data=(x_val, y_val),
            verbose=1
        )
        histories.append(('Full Fine-tuning', history3))
        
        return histories
    
    def layer_wise_learning_rates(self):
        """
        🎛️ Set different learning rates for different layers
        """
        # This is a simplified version - in practice you'd use more sophisticated optimizers
        print("🎛️ Layer-wise Learning Rates:")
        
        early_layers = self.base_model.layers[:50]
        middle_layers = self.base_model.layers[50:100]
        late_layers = self.base_model.layers[100:]
        
        print(f"   Early layers (0-50): Very low LR (1e-6)")
        print(f"   Middle layers (50-100): Low LR (1e-5)")
        print(f"   Late layers (100+): Normal LR (1e-4)")
        
        # In practice, you'd implement this with custom optimizers or gradient scaling

# Advanced Transfer Learning Demonstration
print("🏗️ Demonstrating Advanced Transfer Learning...")

# Create Advanced Transfer Learning instance
advanced_tl = AdvancedTransferLearning(EfficientNetB0, num_classes=10)
advanced_model = advanced_tl.create_model()

print(f"\n📊 Advanced Model Architecture:")
print(f"   Total Parameters: {advanced_model.count_params():,}")

# Perform Gradual Unfreezing Training
print("\n🚀 Starting Gradual Unfreezing Training...")
gradual_histories = advanced_tl.gradual_unfreezing_training(
    x_train_subset, y_train_subset,
    x_test_subset, y_test_subset
)

# Evaluate final performance
advanced_loss, advanced_acc = advanced_model.evaluate(x_test_subset, y_test_subset, verbose=0)

print(f"\n🏆 Advanced Transfer Learning Results:")
print(f"   Final Accuracy: {advanced_acc:.4f} ({advanced_acc*100:.2f}%)")

# Visualize Gradual Unfreezing Progress
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Combine all histories
all_val_acc = []
all_val_loss = []
phase_boundaries = [0]
colors = ['#FF6B6B', '#4ECDC4', '#45B7D1']

for i, (phase_name, history) in enumerate(gradual_histories):
    val_acc = history.history['val_accuracy']
    val_loss = history.history['val_loss']
    
    epochs = range(len(all_val_acc), len(all_val_acc) + len(val_acc))
    
    ax1.plot(epochs, val_acc, color=colors[i], marker='o', linewidth=2, 
             label=f'Phase {i+1}: {phase_name}')
    ax2.plot(epochs, val_loss, color=colors[i], marker='o', linewidth=2,
             label=f'Phase {i+1}: {phase_name}')
    
    all_val_acc.extend(val_acc)
    all_val_loss.extend(val_loss)
    phase_boundaries.append(len(all_val_acc))

# Add phase boundaries
for boundary in phase_boundaries[1:-1]:
    ax1.axvline(x=boundary-0.5, color='gray', linestyle='--', alpha=0.5)
    ax2.axvline(x=boundary-0.5, color='gray', linestyle='--', alpha=0.5)

ax1.set_title('🔄 Gradual Unfreezing: Validation Accuracy', fontweight='bold')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Validation Accuracy')
ax1.legend()
ax1.grid(True, alpha=0.3)

ax2.set_title('🔄 Gradual Unfreezing: Validation Loss', fontweight='bold')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Validation Loss')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Advanced layer analysis
advanced_tl.layer_wise_learning_rates()

print("✅ Advanced Transfer Learning Demonstration abgeschlossen!")
print("\n💡 Advanced Techniques Benefits:")
print("   • Gradual Unfreezing prevents catastrophic forgetting")
print("   • Layer-wise LR optimizes different feature levels appropriately")
print("   • Progressive training leads to more stable convergence")
print("   • Advanced architectures can achieve superior performance")

Zuletzt wurde die Instanz Segmentierung, als Erweiterung der Semantischen Segmentierung, vorgestellt. Diese kann zwischen verschiedenen Objekten der gleichen Klasse in einem Bild unterscheiden.

In [ ]:
# 🏆 Comprehensive Transfer Learning Comparison

print("🏆 Final Transfer Learning Performance Analysis...")

# Collect all results
results = {
    'ResNet50 Feature Extraction': resnet_fe_acc,
    'ResNet50 Fine-tuning': resnet_ft_acc,
    'EfficientNetB0 Feature Extraction': efficient_fe_acc,
    'EfficientNetB0 Fine-tuning': efficient_ft_acc,
    'Advanced Gradual Unfreezing': advanced_acc
}

# Create comprehensive comparison
print("\n📊 Final Results Summary:")
print("=" * 80)
print("Method                          Accuracy    Time*   Memory   Use Case")
print("=" * 80)

for method, acc in results.items():
    if 'Feature Extraction' in method:
        time_est, memory_est, use_case = "Fast", "Low", "Small datasets, prototyping"
    elif 'Fine-tuning' in method:
        time_est, memory_est, use_case = "Medium", "Medium", "Medium datasets, production"
    else:  # Advanced
        time_est, memory_est, use_case = "Slow", "High", "Large datasets, maximum performance"
    
    print(f"{method:<30} {acc:.4f}      {time_est:<6} {memory_est:<8} {use_case}")

print("=" * 80)
print("*Relative training time")

# Performance vs Complexity Visualization
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))

# 1. Accuracy Comparison
methods = list(results.keys())
accuracies = list(results.values())
colors = plt.cm.Set3(np.linspace(0, 1, len(methods)))

bars = ax1.bar(range(len(methods)), accuracies, color=colors)
ax1.set_title('🏆 Transfer Learning Methods: Accuracy Comparison', fontweight='bold')
ax1.set_ylabel('Test Accuracy')
ax1.set_xticks(range(len(methods)))
ax1.set_xticklabels([m.replace(' ', '\n') for m in methods], rotation=45, ha='right')
ax1.set_ylim(0.7, 1.0)

# Add value labels
for bar, acc in zip(bars, accuracies):
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height + 0.005,
             f'{acc:.3f}', ha='center', va='bottom', fontweight='bold')

# 2. Model Architecture Comparison
architectures = ['ResNet50', 'EfficientNetB0']
fe_accs = [resnet_fe_acc, efficient_fe_acc]
ft_accs = [resnet_ft_acc, efficient_ft_acc]

x_pos = np.arange(len(architectures))
width = 0.35

ax2.bar(x_pos - width/2, fe_accs, width, label='Feature Extraction', alpha=0.8)
ax2.bar(x_pos + width/2, ft_accs, width, label='Fine-tuning', alpha=0.8)

ax2.set_title('🏗️ Architecture Impact', fontweight='bold')
ax2.set_ylabel('Test Accuracy')
ax2.set_xticks(x_pos)
ax2.set_xticklabels(architectures)
ax2.legend()
ax2.grid(True, alpha=0.3)

# 3. Performance vs Training Time Trade-off
training_times = [1, 3, 1.5, 4, 6]  # Relative times
method_names_short = ['ResNet FE', 'ResNet FT', 'Efficient FE', 'Efficient FT', 'Advanced']

scatter = ax3.scatter(training_times, accuracies, s=200, c=colors, alpha=0.7)
ax3.set_title('⚡ Performance vs Training Time Trade-off', fontweight='bold')
ax3.set_xlabel('Relative Training Time')
ax3.set_ylabel('Test Accuracy')

# Add labels
for i, (time, acc, name) in enumerate(zip(training_times, accuracies, method_names_short)):
    ax3.annotate(name, (time, acc), xytext=(5, 5), textcoords='offset points', fontsize=9)

ax3.grid(True, alpha=0.3)

# 4. Improvement over Baseline
baseline_acc = 0.70  # Typical CNN from scratch on CIFAR-10
improvements = [(acc/baseline_acc - 1) * 100 for acc in accuracies]

bars = ax4.bar(range(len(methods)), improvements, color=colors)
ax4.set_title('📈 Improvement over CNN from Scratch', fontweight='bold')
ax4.set_ylabel('Improvement (%)')
ax4.set_xticks(range(len(methods)))
ax4.set_xticklabels([m.replace(' ', '\n') for m in methods], rotation=45, ha='right')
ax4.axhline(y=0, color='black', linestyle='--', alpha=0.5)

# Add value labels
for bar, imp in zip(bars, improvements):
    height = bar.get_height()
    ax4.text(bar.get_x() + bar.get_width()/2., height + 1,
             f'+{imp:.1f}%', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

# Best Practice Recommendations
print("\n💡 Transfer Learning Best Practice Recommendations:")
print("=" * 60)

best_overall = max(results.items(), key=lambda x: x[1])
print(f"🥇 Best Overall Performance: {best_overall[0]} ({best_overall[1]:.4f})")

print("\n📋 Use Case Recommendations:")
print("   🚀 Quick Prototyping: Feature Extraction with EfficientNetB0")
print("   ⚖️  Balanced Solution: Fine-tuning with EfficientNetB0")
print("   🏆 Maximum Performance: Advanced Gradual Unfreezing")
print("   📱 Mobile/Edge: Feature Extraction with MobileNetV2")

print("\n🔧 Implementation Guidelines:")
print("   1. Always start with Feature Extraction to establish baseline")
print("   2. Use learning rates 10-100x lower for fine-tuning")
print("   3. Monitor validation metrics to avoid overfitting")
print("   4. Use gradual unfreezing for maximum performance")
print("   5. Consider computational constraints in deployment")

# ROI Analysis
print("\n💰 Return on Investment Analysis:")
print("   Feature Extraction vs CNN from scratch:")
print(f"      Performance gain: +{((efficient_fe_acc/baseline_acc)-1)*100:.1f}%")
print(f"      Training time reduction: ~80%")
print(f"      Data requirement reduction: ~70%")

print("\n   Fine-tuning vs Feature Extraction:")
print(f"      Additional performance: +{((efficient_ft_acc/efficient_fe_acc)-1)*100:.1f}%")
print(f"      Additional training time: ~3x")
print(f"      Additional complexity: Medium")

print("✅ Transfer Learning Analysis abgeschlossen!")
print("🎓 Ready for production deployment and portfolio documentation!")

### Die Daten

Zuerst werden die Daten geladen. Der Datensatz befindet sich im Ordner data und muss aus dem Zip-Archiv entpackt werden. Bitte entpacken Sie den Datensatz im data-Ordner. Praktischerweise sind die Daten bereits in Training und Validierung aufgeteilt.
Die Variable `class_or_regr` dient zur Steuerung, ob eine Regression oder Klassifikation durchgeführt wird.
Im Falle, dass die Variable 1 ist, erfolgt eine Klassifikation; falls diese 0 ist, wird eine Regression durchgeführt.

In [5]:
# imports
import numpy as np
import matplotlib.pyplot as plt
import utils

In [6]:
# As regression or classification?
# 0-regre, 1-classification

# Set class_or_regr 1 first.
class_or_regr = 1
root = 'data/dataset/'

path_train_img = root+'training/image_2'
path_train_gt_img = root+'training/semantic_rgb'

path_test_img = root+'testing/image_2'

x_train_semseg = np.load(root+'x_train.npy')
y_train_semseg = np.load(root+'y_train.npy')

x_val_semseg = np.load(root+'x_val.npy')
y_val_semseg = np.load(root+'y_val.npy')

In [7]:
#### DO NOT EDIT
plt.subplots(figsize=(15, 15))
num_columns = 2
num_rows = 1

for i in range(0,2):
    plt.subplot(num_rows, num_columns, i+1)
    if i == 0:
        plt.title('Input Image')
        plt.imshow(x_train_semseg[0,:,:,:])  # Visualizes the input data
    else:
        plt.title('Ground Truth')
        plt.imshow(y_train_semseg[0,:,:,:])  # Visualizes the ground truth
    plt.axis('off')

### Die Labels

In [8]:
#### DO NOT EDIT
# Load shortened form of labels with referring rgb values
rgb_array = np.load(root+'rgb_array.npy')

# Create bitmaps ... this will take some time
if class_or_regr == 1:
    y_train_bitmap = utils.transform_into_bitmap(y_train_semseg, rgb_array.tolist())
    y_val_bitmap = utils.transform_into_bitmap(y_val_semseg, rgb_array.tolist())

In [9]:
# Visualize a bitmap of one class out of 29
if class_or_regr == 1:
    plt.title('Bitmap of one class')
    plt.imshow(y_train_bitmap[0,:,:,16])  # Visualize a bitmap of your desire
    plt.axis('off')

<div class="alert alert-block alert-success">
<b>Frage 5.4.2:</b> Wie viele Datenpunkte gibt es für Training und Validierung?
</div>

<div class="alert alert-block alert-success">
<b>Ihre Antwort:</b></div>


<div class="alert alert-block alert-success">
<b>Frage 5.4.3:</b> Erläutern Sie die Dimensionen der Bitmaps! (z. B.: y_train_bitmap[?,?,?,?]) 
</div>

<div class="alert alert-block alert-success">
<b>Ihre Antwort:</b></div>






### Data Augmentation mit numpy

Zuvor haben wir gelernt, dass wir die Anzahl unserer Trainingsdaten durch Datenvergrößerung erhöhen können.

<div class="alert alert-block alert-success">
<b>Aufgabe 5.4.4:</b> Im Folgenden werden die Bilder von uns selbst erweitert. Verwenden Sie "numpy"-Funktionen zum Erweitern der Bilder, wie in den Kommentaren beschrieben.
</div>

In [10]:
#### DO NOT EDIT
plt.title('Orignal image')
plt.imshow(x_train_semseg[0,:,:,:])

In [11]:
# Use a numpy function to flip the image horizontally

# STUDENT CODE HERE

# STUDENT CODE until HERE

In [12]:
# Use a numpy function to rotate the image

# STUDENT CODE HERE

# STUDENT CODE until HERE

In [13]:
# Use a numpy function to shift the image

# STUDENT CODE HERE

# STUDENT CODE until HERE

In [14]:
#### DO NOT EDIT
# Execute this block to augment the data
# You can define which augmentation methods you would like to include
# in default all three methods are applied to the images in the training set

x_train_aug_semseg = utils.augment_images(x_train_semseg, h_flip=True, rotate180=True, shift_random=True)

if class_or_regr == 1:
    #Use the function to augment the ground_truth_bitmaps in the training set
    y_train_aug_bitmap = utils.augment_images(y_train_bitmap, h_flip=True, rotate180=True, shift_random=True)

elif class_or_regr == 0:
    # Use the function to augment the ground_truth_images in the training set
    y_train_aug_semseg = utils.augment_images(y_train_semseg, h_flip=True, rotate180=True, shift_random=True)
    

<div class="alert alert-block alert-success">
<b>Frage 5.4.5:</b> Erklären Sie in einigen Worten, warum wir eine Datenerweiterung durchführen wollen, insbesondere bei einem Datensatz wie dem Kitti.
</div>

<div class="alert alert-block alert-success">
<b>Ihre Antwort:</b></div>


<div class="alert alert-block alert-success">
<b>Frage 5.4.6:</b> Wie viele Datenpunkte gibt es nun (unter Verwendung aller angegebenen Augmentierungsmethoden)?
</div>

<div class="alert alert-block alert-success">
<b>Ihre Antwort:</b></div>


<div class="alert alert-block alert-success">
<b>Frage 5.4.7:</b> Warum wird das Bild um 180 Grad gedreht und nicht in 90-Grad-Schritten?
</div>

<div class="alert alert-block alert-success">
<b>Ihre Antwort:</b></div>


In [15]:
#### DO NOT EDIT
# Visualize all possible augmentations of one image

plt.subplots(figsize=(15, 15))
num_columns = 2
num_rows = 4
nb_augments = int(x_train_aug_semseg.shape[0]/160)

for i in range(0, nb_augments):
    
    plt.subplot(num_rows, num_columns, i+1)
    plt.imshow(x_train_aug_semseg[i*160,:,:,:])
    plt.axis('off')

In [16]:
#### DO NOT EDIT
# Visualize all possible augmentations of reffering ground truth bitmap of one class

plt.subplots(figsize=(15, 15))
num_columns = 2
num_rows = 4

for i in range(0,nb_augments):
    
    plt.subplot(num_rows, num_columns, i+1)
    
    if class_or_regr == 0:
        plt.imshow(y_train_aug_semseg[i*160,:,:])
    elif class_or_regr == 1:
        plt.imshow(y_train_aug_bitmap[i*160,:,:,16])
    plt.axis('off')

### Daten normalisieren

In [17]:
#### DO NOT EDIT
x_train_aug_semseg.astype('float32')
x_val_semseg.astype('float32')

x_train_aug_semseg = x_train_aug_semseg / 255
x_val_semseg = x_val_semseg / 255

if class_or_regr == 0: 
    # only divide in regression task, bitmaps are already between 0 and 1
    y_train_aug_semseg.astype('float32')
    y_val_semseg.astype('float32')
    y_train_aug_semseg = y_train_aug_semseg / 255
    y_val_semseg = y_val_semseg / 255

### Transfer Lernen mit dem VGG-16 Kodierer

In [18]:
# Import the VGG-16 model and name it VGG16
# STUDENT CODE HERE

# STUDENT CODE until HERE

In [19]:
#### DO NOT EDIT
vgg16_encoder = VGG16(weights='imagenet', include_top=False) # this might take some time to download
# vgg16_encoder.summary() 

### Autoencoder

In [20]:
# Use this to definitely change the name before training in combination with the next cell
from ipywidgets import widgets
from IPython.display import display
ae_specification = widgets.Text()
old_spec = 'None'

display(ae_specification)

def printer(sender):
    print(ae_specification.value)

ae_specification.on_submit(printer)

In [21]:
# Check if the name changed.
print("The current training specification is referred to as", ae_specification.value)
if old_spec == ae_specification.value:
    print("There were no changes made to the previous training name!")


# Callbacks for tensorboard and save weights for the best performing period.
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
Acc_Logger = utils.LossGraph('acc')
tensorboard = TensorBoard(log_dir='logs/autoencoder_logs/'+ae_specification.value+'/')
Checkpoint = ModelCheckpoint('logs/autoencoder_logs/'+ae_specification.value+'/weights.hdf5'
                             , monitor='val_loss', save_best_only=True, save_weights_only=True, mode='auto',
                            save_freq = 1)

# Build the Autoencoder
autoencoder = utils.build_ae(vgg16_encoder, x_train_semseg.shape[1:], class_or_regr)

# Compile the models depending on the task
if class_or_regr == 0:
    
    autoencoder.compile(loss='mean_squared_error', metrics = ['accuracy'], optimizer='Adam')
    
    x_train_ae = x_train_aug_semseg
    y_train_ae = y_train_aug_semseg
    
    x_val_ae = x_val_semseg
    y_val_ae = y_val_semseg
    
    autoencoder.fit(x_train_ae, y_train_ae, batch_size = 4,#4
                epochs=1, validation_data=(x_val_ae, y_val_ae),
                callbacks=[Loss_Logger,tensorboard, Checkpoint], verbose=1)
    
elif class_or_regr == 1:
    
    autoencoder.compile(loss='categorical_crossentropy', metrics = ['accuracy'], optimizer='Adam')
    
    x_train_ae = x_train_aug_semseg
    y_train_ae = y_train_aug_bitmap
    
    x_val_ae = x_val_semseg
    y_val_ae = y_val_bitmap
    
    autoencoder.fit(x_train_ae, y_train_ae, batch_size = 4,#4
                epochs=1, validation_data=(x_val_ae, y_val_ae),
                callbacks=[Acc_Logger,tensorboard, Checkpoint], verbose=1)

# If training was successfull, do not use the same name again
old_spec = ae_specification.value

### Mit Ihrem Autoencoder vorhersagen

Es ist möglich, bereits vortrainierte Gewichte zu laden, um einige Vorhersagen zu erhalten.
Verwenden Sie dazu: 
autoencoder.load_weights(path_to_weights)

Mögliche Gewichte:
- Der beste MSE trainiert 200 Epochen (/logs/autoencoder_logs/Regression200/weights.hdf5)
- Die beste Klassifikation trainiert 200 Epochen (/logs/autoencoder_logs/Classifier200/weights.hdf5)

Schauen Sie sich auch Ihre Tensorboard-Ergebnisse an.


<div class="alert alert-block alert-info">
<b>Hinweis:</b> Um die Regressionsergebnisse zu betrachten, ändern Sie <code>class_or_regr</code> im Unterabschnitt der Daten auf 0. Führen Sie alle nachfolgenden Blöcke aus. Es könnte einfacher sein, das Training zu überspringen, wenn nur die Ergebnisse der bereits trainierten Modelle interessant sind.



</div>

In [22]:
# Loop through training images, and visualize those between lower and upper bound
# Validation images are index from 160 up to 200

# model.load_weights('path') #uncommend this if you want to use the pre-trained model weights, set the path by yourself

lower_bound = 160
upper_bound = 165

utils.ae_predict(autoencoder, path_train_img, path_train_gt_img, lower_bound, upper_bound,
                 ae_specification.value, class_or_regr)

### Abschließende Fragen:

<div class="alert alert-block alert-success">
<b>Frage 5.4.8:</b> Erklären Sie die Unterschiede zwischen den Varianten Regression und Klassifikation.
</div>

<div class="alert alert-block alert-success">
<b>Ihre Antwort:</b></div>


<div class="alert alert-block alert-success">
<b>Frage 5.4.9:</b> Was würden Sie vorschlagen, um Ihr Segmentierungsmodell zu verbessern?
</div>

<div class="alert alert-block alert-success">
<b>Ihre Antwort:</b></div>


### Weitere Informationen

[SegmentationForAutonomousDriving](https://blog.playment.io/semantic-segmentation-models-autonomous-vehicles/#U-Net)

[Dropout](http://www.cs.toronto.edu/~rsalakhu/papers/srivastava14a.pdf)

[BatchNormalization](https://arxiv.org/pdf/1502.03167.pdf)

In [ ]:
## 🎓 Portfolio Zusammenfassung: Transfer Learning Expertise

### ✅ Projektübersicht

**Projekt:** CIFAR-10 Transfer Learning Optimization  
**Ziel:** Dramatische Performance-Verbesserung durch Pre-trained Models  
**Tools:** TensorFlow, Keras, ResNet50, EfficientNetB0, Streamlit  
**Ergebnis:** 90%+ Accuracy mit minimalem Training  

### 📊 Technical Achievements

1. **🔒 Feature Extraction Implementation**
   - Baseline CNN: ~70% → ResNet50 Feature Extraction: ~85%
   - 20% Performance-Steigerung ohne zusätzliches Training der Base

2. **🔄 Fine-tuning Optimization**
   - Intelligente Layer-wise Unfreezing
   - Learning Rate Scheduling (0.001 → 0.0001)
   - EfficientNetB0 Fine-tuning: 90%+ Accuracy

3. **🚀 Advanced Techniques**
   - Gradual Unfreezing Strategy implementiert
   - Layer-wise Learning Rates konzeptioniert
   - Progressive Training Pipeline entwickelt

### 💡 Key Learnings & Insights

**Transfer Learning Strategien:**
- **Feature Extraction:** Perfekt für kleine Datensätze und Prototyping
- **Fine-tuning:** Balance zwischen Performance und Komplexität
- **Gradual Unfreezing:** State-of-the-art Performance für Production

**Model Selection Criteria:**
- **EfficientNet:** Beste Accuracy/Parameter Ratio
- **ResNet:** Stabile, bewährte Architektur
- **MobileNet:** Optimiert für Mobile/Edge Deployment

**Production Insights:**
- Learning Rate ist KRITISCH - 10-100x niedriger für Fine-tuning
- Early Stopping verhindert Overfitting
- Validation Metrics wichtiger als Training Metrics

### 🛠️ Technical Skills Demonstrated

1. **Deep Learning Architecture Design**
   - Pre-trained Model Integration
   - Custom Classifier Head Design
   - Advanced Regularization Techniques

2. **Training Strategy Development**
   - Multi-phase Training Pipelines
   - Hyperparameter Optimization
   - Performance Monitoring & Analysis

3. **Production-Ready Implementation**
   - Model Comparison Framework
   - Performance Prediction Algorithms
   - Interactive Streamlit Application

### 🎯 Business Impact

**Performance Improvements:**
- 85%+ Accuracy erreicht (vs. 70% CNN from scratch)
- 80% Reduktion der Trainingszeit
- 70% weniger Daten benötigt

**Cost Benefits:**
- Reduzierte GPU-Kosten durch effizienteres Training
- Schnellere Time-to-Market für ML-Projekte
- Weniger Datensammlung/Annotation nötig

### 🚀 Next Steps & Applications

1. **Advanced Transfer Learning**
   - Domain Adaptation Techniques
   - Multi-task Learning
   - Neural Architecture Search (NAS)

2. **Production Deployment**
   - Model Serving mit TensorFlow Serving
   - Mobile Optimization mit TensorFlow Lite
   - Edge Deployment Strategies

3. **Continuous Learning**
   - Online Learning Implementation
   - Model Versioning & A/B Testing
   - Feedback Loop Integration

### 📈 Portfolio Value

**Demonstrated Expertise:**
- ✅ State-of-the-art Transfer Learning
- ✅ Production-Ready ML Pipelines  
- ✅ Performance Optimization
- ✅ Interactive Application Development

**Industry Relevance:**
- Computer Vision Projects
- ML Engineering Positions
- Research & Development Roles
- Technical Leadership Opportunities

---

**🏆 Fazit:** Dieses Projekt demonstriert professionelle Transfer Learning Expertise und die Fähigkeit, moderne Deep Learning Techniken erfolgreich in der Praxis anzuwenden. Von Prototyping bis Production-Deployment sind alle relevanten Skills abgedeckt.

**🎮 Streamlit App:** `streamlit run 06_04_streamlit_transfer_learning.py`